In [5]:
import numpy as np
from astropy.io import ascii
from scipy.optimize import root_scalar
from astropy import units as u
from astropy import constants as const
from astropy.time import Time
import matplotlib.pyplot as plt

"""
a = semieje mayor
e = excentricidad
omega = longitud del nodo ascendente
w = argumento del pericentro
i = inclinacion de la orbita
n = movimiento medio
lo = anomalia media en la epoca
tp = tiempo del perihelio
to = tiempo de la epoca
M = la suma de las masas de los cuerpos
"""

# Ecuacion de kepler
def Kepler(E,l,ecc):
    f = E - ecc*np.sin(E)-l
    return f



#Orbita de la tierra
to = Time('2023-07-09T00:00:00', scale='tdb', format = 'isot')
#to = to.utc

#Efemerides Tierra
a = (1.495582533630905E+8*u.km).to('au')
ecc = 1.694863932474438E-02
omega = 1.498625963929686E+02*u.deg
w = 3.146587763491455E+02*u.deg
iota = 4.164075603038432E-03*u.deg 
n = 1.141204629731537E-05*u.deg/u.s
lo = 1.817846947871890E+02*u.deg

#Transformacion de unidades
iota = iota.to('rad')
omega = omega.to('rad')
w = w.to('rad')
n = n.to('rad/s')
lo = lo.to('rad')


print("xd")





xd
